### 1. Defining the Question

##### a) Specifying the Question

> You have been recruited as a football analyst in a company - Mchezopesa Ltd and tasked to make a prediction result of a game between team 1 and team 2, based on who's home and who's away, and on whether or not the game is friendly

##### b) Defining the Metric for Success

> The project will be considered a success if the models created are able to predict results of a game to an accuracy of at least 80%

##### c) Understanding the Context

> FIFA is a non-profit organization that describes itself as an international governing body of association football, futsal and beach soccer. It is the highest governing body of association football. It is responsible for the rules of association football, the regulation of the sport, the promotion of football across the world.


> We are supposed to predict the result of a game between team 1 and 2 based on who's home and who's away, and on whether or not the game is friendly


> This will be done by using machine learning models to calculate the points of each team and predict the possible winner of the game

##### d) Recording the Experimental Design

> Perform your EDA

> Perform any necessary feature engineering 

> Check of multicollinearity

> Start building the model

> Cross-validate the model

> Compute RMSE

> Create residual plots for your models, and assess their heteroscedasticity using Bartlett’s test

> Perform appropriate regressions on the data including your justification

> Challenge your solution by providing insights on how you can make improvements.

##### e) Data Relevance

> The data provided for this project is relevant and has been provided by Mchezopesa Ltd and is available on their website.

### 2. Reading Data

In [105]:
# let's import the libraries we need to use for the project
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, LabelEncoder
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, f1_score, precision_score, recall_score, classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RepeatedKFold
from scipy.stats import bartlett, chi2, loguniform
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression, Ridge, LogisticRegression, RidgeClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut

In [110]:
# let's import the datasets
results_df = pd.read_csv('results.csv')

In [111]:
# let's import the second dataset
fifa_df = pd.read_csv('fifa_ranking.csv')

### 3. Checking Data

In [112]:
# let's preview the datasets
results_df.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


> These are the top 5 rows of the first dataset.

In [113]:
# let's preview the datasets
fifa_df.head()

,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date
0,1,Germany,GER,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
1,2,Italy,ITA,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
2,3,Switzerland,SUI,0.0,50,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
3,4,Sweden,SWE,0.0,55,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
4,5,Argentina,ARG,0.0,51,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08


> These are the top 5 rows of the second dataset.

In [114]:
# let's check the tail of the datasets
results_df.tail()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
40834,2019-07-18,American Samoa,Tahiti,8,1,Pacific Games,Apia,Samoa,True
40835,2019-07-18,Fiji,Solomon Islands,4,4,Pacific Games,Apia,Samoa,True
40836,2019-07-19,Senegal,Algeria,0,1,African Cup of Nations,Cairo,Egypt,True
40837,2019-07-19,Tajikistan,North Korea,0,1,Intercontinental Cup,Ahmedabad,India,True
40838,2019-07-20,Papua New Guinea,Fiji,1,1,Pacific Games,Apia,Samoa,True


> These are the last 5 rows of the first dataset.

In [116]:
# let's check the tail of the datasets
fifa_df.tail()

,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date
57788,206,Anguilla,AIA,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONCACAF,2018-06-07
57789,206,Bahamas,BAH,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONCACAF,2018-06-07
57790,206,Eritrea,ERI,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,2018-06-07
57791,206,Somalia,SOM,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,2018-06-07
57792,206,Tonga,TGA,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,OFC,2018-06-07


> These are the last 5 rows of the second dataset.

In [120]:
# let's use a print function to check the shape of the datasets
print('The results dataset has {} rows and {} columns.'.format(results_df.shape[0], results_df.shape[1]))


The results dataset has 40839 rows and 9 columns.


> The first dataset has 40839 rows and 9 columns.

In [121]:
# let's use a print function to check the shape of the datasets
print('The fifa dataset has {} rows and {} columns.'.format(fifa_df.shape[0], fifa_df.shape[1]))

The fifa dataset has 57793 rows and 16 columns.


> The second dataset has 57793 rows and 16 columns.

In [122]:
# let's check the data types of the datasets
results_df.dtypes

date          object
home_team     object
away_team     object
home_score     int64
away_score     int64
tournament    object
city          object
country       object
neutral         bool
dtype: object

> These are the columns of the first dataset.

In [123]:
# let's check the data types of the datasets
fifa_df.dtypes

rank                         int64
country_full                object
country_abrv                object
total_points               float64
previous_points              int64
rank_change                  int64
cur_year_avg               float64
cur_year_avg_weighted      float64
last_year_avg              float64
last_year_avg_weighted     float64
two_year_ago_avg           float64
two_year_ago_weighted      float64
three_year_ago_avg         float64
three_year_ago_weighted    float64
confederation               object
rank_date                   object
dtype: object

> These are the columns of the second dataset.

In [124]:
# let's see a description of the datasets
results_df.describe()

,home_score,away_score
count,40839.000000,40839.000000
mean,1.745709,1.188105
std,1.749145,1.405120
min,0.000000,0.000000
25%,1.000000,0.000000
50%,1.000000,1.000000
75%,2.000000,2.000000
max,31.000000,21.000000


In [125]:
# let's see a description of the datasets
fifa_df.describe()

,rank,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted
count,57793.000000,57793.000000,57793.000000,57793.000000,57793.000000,57793.000000,57793.000000,57793.000000,57793.000000,57793.000000,57793.000000,57793.000000
mean,101.628086,122.068637,332.302926,-0.009897,61.798602,61.798602,61.004602,30.502377,59.777462,17.933277,59.173916,11.834811
std,58.618424,260.426863,302.872948,5.804309,138.014883,138.014883,137.688204,68.844143,136.296079,40.888849,135.533343,27.106675
min,1.000000,0.000000,0.000000,-72.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,51.000000,0.000000,56.000000,-2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,101.000000,0.000000,272.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,152.000000,92.790000,525.000000,1.000000,32.250000,32.250000,26.660000,13.330000,21.500000,6.450000,21.250000,4.250000
max,209.000000,1775.030000,1920.000000,92.000000,1158.660000,1158.660000,1169.570000,584.790000,1159.710000,347.910000,1200.770000,240.150000


### 4. External Data Source Validation

> The teams that are playing in the game have been verified from the FIFA website and the matches actually played.

### 5. Tidying the Dataset

In [126]:
# let's check for null values
results_df.isnull().sum().any()

False

> The first dataset does not have any null values

In [127]:
# let's check for null values
fifa_df.isnull().sum().any()

False

> The second dataset has no null values

In [129]:
# let's check for anomalies
results_df.duplicated().sum().any()

False

> There is no duplicate data in the first dataset

In [130]:
# let's check for anomalies
fifa_df.duplicated().sum().any()

True

In [132]:
# let's drop the duplicated data
fifa_df.drop_duplicates(inplace=True)
# checking for duplicated data again
fifa_df.duplicated().sum().any()

False

> The second dataset however has some duplicate data

In [142]:
# Outliers
# let's create a function that checks for the percentage of outliers in a dataset using Interquartile range
def check_outliers(df):
    # instantiate the quantile variables
    Quantile_1, Quantile_2, Quantile_3 = 0, 0, 0

    # converting to dataframe
    check_outliers = pd.DataFrame()

    # let's get the numerical columns
    num_cols = df.select_dtypes(include=['int64', 'float64'])

    # getting the quantiles from the numerical columns
    Quantile_1 = num_cols.quantile(0.25)
    Quantile_3 = num_cols.quantile(0.75)

    # let's get the IQR
    IQR = Quantile_3 - Quantile_1

    # let's get the outliers
    check_outliers = num_cols[((num_cols < (Quantile_1 - 1.5 * IQR)) | (num_cols > (Quantile_3 + 1.5 * IQR))).any(axis=1)]

    # printing the number of outliers and the percentage of outliers
    print('There are {} outliers in the dataset.'.format(check_outliers.shape[0]))
    print('The percentage of outliers in the dataset is {}%.'.format(round((check_outliers.shape[0] / df.shape[0]) * 100, 2)))

    # reusable #

In [143]:
# outliers in the results dataset
check_outliers(results_df)

There are 5683 outliers in the dataset.
The percentage of outliers in the dataset is 13.92%.


In [144]:
# outliers in the fifa dataset
check_outliers(fifa_df)

There are 21235 outliers in the dataset.
The percentage of outliers in the dataset is 36.77%.


> We wont be dropping any outliers for both datasets as these are crucial datasets

In [156]:
# let's copy the fifa dataset to a new dataset in order to ranks for the away team during merging
fifa_df_copy = fifa_df.copy()

# let's merge the two datasets
main_merged = pd.merge(pd.merge(results_df, fifa_df, left_on=['date', 'home_team'], how = 'inner', right_on = ['rank_date', 'country_full']), fifa_df_copy, left_on=['date', 'away_team'], how = 'inner', right_on = ['rank_date', 'country_full'])

In [160]:
# let's merge the two fifa dataframes using an inner join
merged_df = pd.merge(pd.merge(results_df, fifa_df[['rank', 'country_full', 'rank_date']], how='inner', 
            left_on=['date', 'home_team'], 
            right_on=['rank_date', 'country_full']), fifa_df_copy[['rank', 'country_full', 'rank_date']], how='inner',
            left_on=['date', 'away_team'],
            right_on=['rank_date', 'country_full'])

# let's check the shape of the merged dataframe
merged_df.shape

# preview the merged dataframe
merged_df.head()

# let's drop the duplicate columns
merged_df.drop(['rank_date_x', 'rank_date_y', 'country_full_x', 'country_full_y', 'neutral', 'country', 'city'], axis=1, inplace=True)

# let's rename the rank_x and rank_y columns to home_team_rank and away_team_rank
merged_df.rename(columns={'rank_x': 'home_team_rank', 'rank_y': 'away_team_rank'}, inplace=True)

# preview the merged dataframe
merged_df.head(5)

,date,home_team,away_team,home_score,away_score,tournament,home_team_rank,away_team_rank
0,1993-08-08,Bolivia,Uruguay,3,1,FIFA World Cup qualification,59,22
1,1993-08-08,Brazil,Mexico,1,1,Friendly,8,14
2,1993-08-08,Ecuador,Venezuela,5,0,FIFA World Cup qualification,35,94
3,1993-08-08,Guinea,Sierra Leone,1,0,Friendly,65,86
4,1993-08-08,Paraguay,Argentina,1,3,FIFA World Cup qualification,67,5


In [164]:
# let's add a results column to the merged dataframe to help us 
# compute the points and to know whether or not a team won by appending a W for won, a D for draw and a L for lost based on the home team's goals
merged_df['results'] = merged_df.apply(lambda row: 'W' if row['home_score'] > row['away_score'] else ('D' if row['home_score'] == row['away_score'] else 'L'), axis=1)

# preview the changes to dataframe
merged_df.tail(5)

,date,home_team,away_team,home_score,away_score,tournament,home_team_rank,away_team_rank,results
866,2018-06-07,England,Costa Rica,2,0,Friendly,12,23,W
867,2018-06-07,Uruguay,Uzbekistan,3,0,Friendly,14,95,W
868,2018-06-07,Portugal,Algeria,3,0,Friendly,4,66,W
869,2018-06-07,Iceland,Ghana,2,2,Friendly,22,47,D
870,2018-06-07,India,New Zealand,1,2,Intercontinental Cup,97,120,L


### 6. Exploratory Analysis

In [ ]:
# Ploting the univariate summaries and recording our observations
#

### 7. Implementing the Solution

In [ ]:
# Implementing the Solution
# 

### 8. Challenging the solution

In [ ]:
# Reviewing the Solution 
#

### 9. Follow up questions

##### a). Did we have the right data?

##### b). Do we need other data to answer our question?

##### c). Did we have the right question?